In [2]:
import pandas as pd

In [3]:
cancer_data = pd.read_csv('/content/drive/MyDrive/Dr. V/MSK-CHORD/Individual_cancer_type_data/Pancreatic_Cancer.csv')

In [4]:
print(len(cancer_data['SAMPLE_ID'].unique()))
cancer_data.shape
print(len(cancer_data['PATIENT_ID']))

3109
14838


In [5]:
cancer_data.columns

Index(['SAMPLE_ID', 'PATIENT_ID', 'CANCER_TYPE', 'CANCER_TYPE_DETAILED',
       'Hugo_Symbol', 'Consequence', 'Reference_Allele', 'Tumor_Seq_Allele1',
       'Tumor_Seq_Allele2', 'Tumor_Sample_Barcode', 'HGVSp'],
      dtype='object')

In [6]:
total_patients = len(cancer_data['SAMPLE_ID'].unique())
most_frequent_symbol = cancer_data.groupby('Hugo_Symbol')['SAMPLE_ID'].nunique()
mutation_frequency = ((most_frequent_symbol / total_patients) * 100).round(2)
frequency_df = pd.DataFrame({'most_freq_symbol': most_frequent_symbol,'Mutation_Frequency': mutation_frequency})
sorted_mutation_frequency = frequency_df.sort_values(by='Mutation_Frequency', ascending=False)
top_10_mutated_genes = sorted_mutation_frequency.head(10)
#.drop("CDKN2Ap16INK4A")
top_10 = top_10_mutated_genes.drop(columns=['most_freq_symbol'])
#sort_frequency = most_frequent_symbol.sort_values(ascending=False)
#top_10 = sort_frequency.head(10)
#top_10
genes_list = list(top_10.index)
#print('genes list', genes_list)
gene_df = {}
for gene in genes_list:
  filter_hugo = cancer_data[cancer_data['Hugo_Symbol'] == gene]
  filtered_cancer_data = cancer_data[(cancer_data['Hugo_Symbol']== gene)& (cancer_data['Consequence']== 'missense_variant')]
  gene_df[gene] = filtered_cancer_data
print(top_10_mutated_genes)
print(gene_df.keys())

             most_freq_symbol  Mutation_Frequency
Hugo_Symbol                                      
KRAS                     2519               81.02
TP53                     2077               66.81
CDKN2A                    632               20.33
SMAD4                     607               19.52
ARID1A                    249                8.01
RNF43                     182                5.85
MEN1                      130                4.18
KMT2D                     121                3.89
KDM6A                     117                3.76
TGFBR2                    115                3.70
dict_keys(['KRAS', 'TP53', 'CDKN2A', 'SMAD4', 'ARID1A', 'RNF43', 'MEN1', 'KMT2D', 'KDM6A', 'TGFBR2'])


In [7]:
!pip install unipressed

In [8]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 28.6 MB/s eta 0:00:00


In [9]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 3.7 MB/s eta 0:00:00


In [10]:
#can delete after script is ready
print(genes_list)

['KRAS', 'TP53', 'CDKN2A', 'SMAD4', 'ARID1A', 'RNF43', 'MEN1', 'KMT2D', 'KDM6A', 'TGFBR2']


In [ ]:
#gene_decoy = ['KRAS','APC']

In [11]:
#can delete after script is ready
from unipressed import IdMappingClient
import time
request = IdMappingClient.submit(
    source="GeneCards", dest="UniProtKB", ids= genes_list
)
time.sleep(1.0)
uniprotid = list(request.each_result())

In [12]:
#print(uniprotid)
gene_to_uniprot = {entry['from']: entry['to'] for entry in uniprotid}
print(gene_to_uniprot)

{'KRAS': 'P01116', 'TP53': 'P04637', 'CDKN2A': 'Q8N726', 'SMAD4': 'Q13485', 'ARID1A': 'O14497', 'RNF43': 'Q68DV7', 'MEN1': 'O00255', 'KMT2D': 'O14686', 'KDM6A': 'O15550', 'TGFBR2': 'P37173'}


Getting Uniprot 1, 2 and source column: To filter out the PPi with PDB

In [13]:
from Bio import ExPASy
from Bio import SwissProt

ppi_data = pd .read_csv('/content/drive/MyDrive/Dr. V/pioneer_human.txt', sep='\t', header= 0)
#print(ppi_data.columns)
df_filtered = ppi_data[['uniprot1', 'uniprot2','source', 'interface_residues1','interface_residues2']]
print(ppi_data.shape)

(146138, 7)


In [14]:
pioneer_data = {}
for gene_name in genes_list:
  filtered_rows = df_filtered[(df_filtered['uniprot1'] == gene_to_uniprot[gene_name]) &
                              (df_filtered['source'] == 'PDB')]

  entry_name = []
  gene_id = []
  for num in filtered_rows['uniprot2']:
      record = ExPASy.get_sprot_raw(num)
      record = SwissProt.read(record)
      entry_name.append(record.entry_name)
      gene_id.append(record.gene_name)
  filtered_rows['entry_name'] = entry_name
  #filtered_rows['gene_id'] = gene_id
  pioneer_data[gene_name] = filtered_rows


<ipython-input-14-a8b4644ebee2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['entry_name'] = entry_name
<ipython-input-14-a8b4644ebee2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['entry_name'] = entry_name
<ipython-input-14-a8b4644ebee2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [15]:
print(pioneer_data.keys())

dict_keys(['KRAS', 'TP53', 'CDKN2A', 'SMAD4', 'ARID1A', 'RNF43', 'MEN1', 'KMT2D', 'KDM6A', 'TGFBR2'])


In [ ]:
#print( pioneer_data.keys())
pioneer_data['TP53']

#for gene, df in pioneer_data.items():
    #pioneer_data[gene] = df.drop(columns=['gene_id'], errors='ignore')


,uniprot1,uniprot2,source,interface_residues1,interface_residues2,entry_name
1176,P04637,Q13625,PDB,"[165,167,168,171,178,179,181,182,183,239,241,2...","[1021,1023,1024,1025,1026,1071,1072,1073,1074,...",ASPP2_HUMAN
1266,P04637,Q12888,PDB,"[101,102,103,104,105,106,107,108,148,149,150,1...","[1495,1497,1498,1499,1500,1502,1521,1522,1523,...",TP53B_HUMAN
1715,P04637,P31947,PDB,"[382,383,384,385,386,388,389,390,391,392,393]","[42,45,46,49,50,53,54,56,57,59,60,63,64,67,122...",1433S_HUMAN
1787,P04637,Q96EB6,PDB,"[379,380,381,383,385]","[414,415,416,417,420,423,444,445,446,447,450]",SIR1_HUMAN
2522,P04637,Q09472,PDB,"[8,9,10,11,13,15,17,18,19,21,22,23,25,26,27,28...","[1345,1347,1350,1351,1386,1724,1725,1727,1729,...",EP300_HUMAN
3124,P04637,Q93009,PDB,"[358,359,360,361,362,363,364,365,366,367]","[102,104,152,153,162,163,164,165,166,167,168]",UBP7_HUMAN
3542,P04637,P20248,PDB,"[378,379,380,381,382,383,384,385,386]","[210,213,217,220,223,224,250,254,280,281,283,285]",CCNA2_HUMAN
3973,P04637,P53803,PDB,"[183,185,187]",[31],RPAB4_HUMAN
4344,P04637,Q9NRG4,PDB,"[368,369,370,371,372,373,374,375]","[105,180,181,183,184,185,187,191,214,215,239,2...",SMYD2_HUMAN
4417,P04637,P24928,PDB,"[12,13,14,15,16,18,19,20,21,23,24,28,30,38,39,...","[149,150,151,152,163,164,165,166,167,168,172,1...",RPB1_HUMAN


In [16]:
for gene, df in gene_df.items():
  cols = [col for col in filtered_cancer_data.columns
          if col.endswith('_ID') or col in ['Hugo_Symbol','Consequence','HGVSp']]
  cancer_mut = df[cols]
  total_mut = len(cancer_mut)
  mut_count = cancer_mut['HGVSp'].value_counts()
  mut_percentage = ((mut_count/total_mut)*100).round(2)
  cancer_mut.loc[:, ['mutation_percentage']] = cancer_data['HGVSp'].map(mut_percentage)
  mut_count_table = pd.DataFrame({'mutation_count': mut_count, 'mutation_percentage': mut_percentage})
  filtered_rows = pioneer_data[gene]
  with pd.ExcelWriter(f"/content/drive/MyDrive/Dr. V/MSK-CHORD/MSK-CHORD-top_10/Pancreatic_cancer_top_10/{gene}_missense_variants.xlsx", engine='xlsxwriter') as writer:
        cancer_mut.to_excel(writer, sheet_name='mutations', index=False)
        mut_count_table.to_excel(writer, sheet_name='AA_change_freq%', index=True)
        filtered_rows.to_excel(writer, sheet_name="Pioneer_PPI's", index=False)